In [2]:
!pip install --upgrade pip
!pip install soundfile
!pip install librosa
!pip install transformers==4.39.3
!pip install git+https://github.com/ai4bharat/IndicF5.git
!pip install vocos


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Cloning https://github.com/ai4bharat/IndicF5.git to /tmp/pip-req-build-t7ppaq07
  Running command git clone --filter=blob:none --quiet https://github.com/ai4bharat/IndicF5.git /tmp/pip-req-build-t7ppaq07
  Resolved https://github.com/ai4bharat/IndicF5.git to commit 13f7c4d627cc10111aea8fe9c0039462cacacdc7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
from transformers import AutoProcessor, VQGanVAEModel

# vocos enhancement
from huggingface_hub import hf_hub_download

vocos_path = hf_hub_download(repo_id="charactr/vocos-mel-24khz", filename="vocos.pt")
print("Vocos downloaded:", vocos_path)


ImportError: cannot import name 'VQGanVAEModel' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HUGGINGFACE_TOKEN')  # or your secret variable name
login(token=hf_token)

In [ ]:
import os

# Path to your folder inside Google Drive
BASE_DIR = "/content/drive/MyDrive/IndicF5"

REF_AUDIO_PATH = os.path.join(BASE_DIR, "prompts/ref.wav")
REF_TEXT_PATH  = os.path.join(BASE_DIR, "prompts/ref.txt")

# Load reference text
with open(REF_TEXT_PATH, "r", encoding="utf-8") as f:
    REF_TEXT = f.read().strip()

print("Reference Audio:", REF_AUDIO_PATH)
print("Reference Text:", REF_TEXT)


In [6]:
from transformers import AutoModel

repo_id = "ai4bharat/IndicF5"

model = AutoModel.from_pretrained(
    repo_id,
    trust_remote_code=True
)

print("✅ Model Loaded Successfully!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /t

Word segmentation module jieba initialized.



/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ValueError: The model class you are passing has a `config_class` attribute that is not consistent with the config class you passed (model has <class 'transformers_modules.ai4bharat.IndicF5.b82d286220e3070e171f4ef4b4bd047b9a447c9a.model.INF5Config'> and you passed <class 'transformers_modules.ai4bharat.IndicF5.b82d286220e3070e171f4ef4b4bd047b9a447c9a.model.INF5Config'>. Fix one of those so they match!

In [ ]:
import numpy as np
import soundfile as sf

# Your input text (Tamil example—you can change)
input_text = "வணக்கம்! இது ஒரு தமிழ் உரை உருவாக்கும் சோதனை."

print("Generating audio...")

audio = model(
    input_text,
    ref_audio_path=REF_AUDIO_PATH,
    ref_text=REF_TEXT
)

# Normalize if needed
if audio.dtype == np.int16:
    audio = audio.astype(np.float32) / 32768.0

OUTPUT_PATH = os.path.join(BASE_DIR, "output/output.wav")
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

sf.write(OUTPUT_PATH, audio.astype(np.float32), samplerate=24000)

print("Saved:", OUTPUT_PATH)


In [ ]:
from IPython.display import Audio
Audio(OUTPUT_PATH)